In [39]:
import pandas as pd
dict={"userId":[12,23,12,23,67,63,78],"contentId":[123,234,657,123,890,657,890],"eventStrength":[5,3,2,7,8,9,2],"Name":["Python","Java","backend","frontend","AI","ML","Ds"]}
df=pd.DataFrame(dict)
print(type(df))
grouped=df.groupby(["userId","contentId"])["eventStrength"].sum().reset_index()
grouped=grouped.set_index("userId")
grouped
df.groupby(["userId","contentId"]).size().groupby("userId").size()

<class 'pandas.core.frame.DataFrame'>


userId
12    2
23    2
63    1
67    1
78    1
dtype: int64

In [81]:

import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import scipy
import nltk
import time
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix
from nltk.corpus import stopwords



In [2]:
articles_df=pd.read_csv("recommenderDataset/shared_articles.csv")
articles_df=articles_df[articles_df["eventType"]=="CONTENT SHARED"]
articles_df.head(10)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en
5,1459194522,CONTENT SHARED,-2826566343807132236,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.coindesk.com/ieee-blockchain-oxford...,IEEE to Talk Blockchain at Cloud Computing Oxf...,One of the largest and oldest organizations fo...,en
6,1459194557,CONTENT SHARED,-2148899391355011268,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.newsbtc.com/2016/03/28/banks-need-c...,Banks Need To Collaborate With Bitcoin and Fin...,It will take time until banks come around to t...,en
7,1459194599,CONTENT SHARED,4119190424078847945,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/blockchai...,Blockchain Technology Could Put Bank Auditors ...,When most people think about computers and rob...,en
8,1459194751,CONTENT SHARED,-7926018713416777892,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://news.bitcoin.com/conglomerates-intervi...,Why Decentralized Conglomerates Will Scale Bet...,"Bitcoin.com spoke with the OpenLedger CEO, Ron...",en
9,1459194842,CONTENT SHARED,3353902017498793780,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://www.cryptocoinsnews.com/ethereum-rise-...,The Rise And Growth of Ethereum Gets Mainstrea...,"Ethereum, considered by many to be the most pr...",en
10,1459210504,CONTENT SHARED,-9157338616628196758,5206835909720479405,-7864441319395545950,NaN,NaN,NaN,HTML,http://economia.ig.com.br/2016-03-27/situacao-...,Situação financeira ruim de varejistas pressio...,A queda nas vendas e a deterioração na situaçã...,pt


In [3]:
interactions_df = pd.read_csv('recommenderDataset/users_interactions.csv')
interactions_df.head(10)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN
5,1465413742,VIEW,310515487419366995,-8763398617720485024,1395789369402380392,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,MG,BR
6,1465415950,VIEW,-8864073373672512525,3609194402293569455,1143207167886864524,NaN,NaN,NaN
7,1465415066,VIEW,-1492913151930215984,4254153380739593270,8743229464706506141,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR
8,1465413762,VIEW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
9,1465413771,VIEW,3064370296170038610,3609194402293569455,1143207167886864524,NaN,NaN,NaN


# Data munging
As there are different interactions types, 
we associate them with a weight or strength, assuming that, <br>for example, a comment in an article indicates a higher interest of the user on the item than a like, or than a simple view.

In [4]:
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

interactions_df['eventStrength'] = interactions_df['eventType'].apply(lambda x: event_type_strength[x])


Recommender systems have a problem known as user cold-start, <br>in which is hard do provide personalized recommendations for users with none or a very few number of consumed items,<br> due to the lack of information to model their preferences.
For this reason, we are keeping in the dataset only users with at leas 5 interactions.

In [5]:
users_interactions_count_df = interactions_df.groupby(['personId', 'contentId']).size().groupby('personId').size()
print('# users: %d' % len(users_interactions_count_df))
users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]
print('# users with at least 5 interactions: %d' % len(users_with_enough_interactions_df))


# users: 1895
# users with at least 5 interactions: 1140


In [6]:
print('# of interactions: %d' % len(interactions_df))
interactions_from_selected_users_df = interactions_df.merge(users_with_enough_interactions_df, 
               how = 'right',
               left_on = 'personId',
               right_on = 'personId')
print('# of interactions from users with at least 5 interactions: %d' % len(interactions_from_selected_users_df))

# of interactions: 72312
# of interactions from users with at least 5 interactions: 69868


In [7]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = interactions_from_selected_users_df \
                    .groupby(['personId', 'contentId'])['eventStrength'].sum() \
                    .apply(smooth_user_preference).reset_index()
print('# of unique user/item interactions: %d' % len(interactions_full_df))
interactions_full_df.head(10)

# of unique user/item interactions: 39106


,personId,contentId,eventStrength
0,-9223121837663643404,-8949113594875411859,1.000000
1,-9223121837663643404,-8377626164558006982,1.000000
2,-9223121837663643404,-8208801367848627943,1.000000
3,-9223121837663643404,-8187220755213888616,1.000000
4,-9223121837663643404,-7423191370472335463,3.169925
5,-9223121837663643404,-7331393944609614247,1.000000
6,-9223121837663643404,-6872546942144599345,1.000000
7,-9223121837663643404,-6728844082024523434,1.000000
8,-9223121837663643404,-6590819806697898649,1.000000
9,-9223121837663643404,-6558712014192834002,1.584963


# Evaluation
Evaluation is important for machine learning projects, <br>
because it allows to compare objectivelly different algorithms and hyperparameter choices for models.
One key aspect of evaluation is to ensure that the trained model generalizes <br>
for data it was not trained on, using Cross-validation techniques.<br> 
We are using here a simple cross-validation approach named holdout, <br>
in which a random data sample (20% in this case) are kept aside in the training process,<br> 
and exclusively used for evaluation. All evaluation metrics reported here are computed using the test set.

Ps. A more robust evaluation approach could be to split train and test sets by a reference date,<br> 
where the train set is composed by all interactions before that date, and the test set are interactions <br>after that date. For the sake of simplicity, we chose the first random approach for this notebook, but you may want to try the second approach to better simulate how the recsys would perform in production predicting "future" users interactions.

In [8]:
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['personId'], 
                                   test_size=0.20,
                                   random_state=42)

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

# interactions on Train set: 31284
# interactions on Test set: 7822


In [9]:
#Indexing by personId to speed up the searches during evaluation
interactions_full_indexed_df = interactions_full_df.set_index('personId')
interactions_train_indexed_df = interactions_train_df.set_index('personId')
interactions_test_indexed_df = interactions_test_df.set_index('personId')

In [10]:

def get_items_interacted(person_id, interactions_df):
    # Get the user's data and merge in the movie information.
    interacted_items = interactions_df.loc[person_id]['contentId']
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])
#Top-N accuracy metrics consts
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100

class ModelEvaluator:


    def get_not_interacted_items_sample(self, person_id, sample_size, seed=42):
        interacted_items = get_items_interacted(person_id, interactions_full_indexed_df)
        all_items = set(articles_df['contentId'])
        non_interacted_items = all_items - interacted_items

        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)

    def _verify_hit_top_n(self, item_id, recommended_items, topn):        
            try:
                index = next(i for i, c in enumerate(recommended_items) if c == item_id)
            except:
                index = -1
            hit = int(index in range(0, topn))
            return hit, index

    def evaluate_model_for_user(self, model, person_id):
        #Getting the items in test set
        interacted_values_testset = interactions_test_indexed_df.loc[person_id]
        if type(interacted_values_testset['contentId']) == pd.Series:
            person_interacted_items_testset = set(interacted_values_testset['contentId'])
        else:
            person_interacted_items_testset = set([int(interacted_values_testset['contentId'])])  
        interacted_items_count_testset = len(person_interacted_items_testset) 

        #Getting a ranked recommendation list from a model for a given user
        person_recs_df = model.recommend_items(person_id, 
                                               items_to_ignore=get_items_interacted(person_id, 
                                                                                    interactions_train_indexed_df), 
                                               topn=10000000000)

        hits_at_5_count = 0
        hits_at_10_count = 0
        #For each item the user has interacted in test set
        for item_id in person_interacted_items_testset:
            #Getting a random sample (100) items the user has not interacted 
            #(to represent items that are assumed to be no relevant to the user)
            non_interacted_items_sample = self.get_not_interacted_items_sample(person_id, 
                                                                          sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS, 
                                                                          seed=item_id%(2**32))

            #Combining the current interacted item with the 100 random items
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

            #Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = person_recs_df[person_recs_df['contentId'].isin(items_to_filter_recs)]                    
            valid_recs = valid_recs_df['contentId'].values
            #Verifying if the current interacted item is among the Top-N recommended items
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10

        #Recall is the rate of the interacted items that are ranked among the Top-N recommended items, 
        #when mixed with a set of non-relevant items
        recall_at_5 = hits_at_5_count / float(interacted_items_count_testset)
        recall_at_10 = hits_at_10_count / float(interacted_items_count_testset)

        person_metrics = {'hits@5_count':hits_at_5_count, 
                          'hits@10_count':hits_at_10_count, 
                          'interacted_count': interacted_items_count_testset,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        return person_metrics

    def evaluate_model(self, model):
        #print('Running evaluation for users')
        people_metrics = []
        for idx, person_id in enumerate(list(interactions_test_indexed_df.index.unique().values)):
            #if idx % 100 == 0 and idx > 0:
            #    print('%d users processed' % idx)
            person_metrics = self.evaluate_model_for_user(model, person_id)  
            person_metrics['_person_id'] = person_id
            people_metrics.append(person_metrics)
        print('%d users processed' % idx)

        detailed_results_df = pd.DataFrame(people_metrics) \
                            .sort_values('interacted_count', ascending=False)
        
        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        
        global_metrics = {'modelName': model.get_model_name(),
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}    
        return global_metrics, detailed_results_df
    
model_evaluator = ModelEvaluator() 

# Popularity model
A common (and usually hard-to-beat) baseline approach is the Popularity model.<br> 
This model is not actually personalized - it simply recommends to <br>
a user the most popular items that the user has not previously consumed.<br> 
As the popularity accounts for the "wisdom of the crowds", <br>
it usually provides good recommendations, generally interesting for most people.
Ps. The main objective of a recommender system is to leverage the long-tail items to <br>
the users with very specific interests, which goes far beyond this simple technique.

In [11]:
#Computes the most popular items
item_popularity_df = interactions_full_df.groupby('contentId')['eventStrength']\
                                    .sum()\
                                    .sort_values(ascending=False)\
                                    .reset_index()
item_popularity_df.head(10)


,contentId,eventStrength
0,-4029704725707465084,307.733799
1,-6783772548752091658,233.762157
2,-133139342397538859,228.024567
3,-8208801367848627943,197.107608
4,-6843047699859121724,193.825208
5,8224860111193157980,189.044680
6,-2358756719610361882,183.110951
7,2581138407738454418,180.282876
8,7507067965574797372,179.094002
9,1469580151036142903,170.548969


In [12]:

class PopularityRecommender:
    
    MODEL_NAME = 'Popularity'
    
    def __init__(self, popularity_df, items_df=None):
        self.popularity_df = popularity_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Recommend the more popular items that the user hasn't seen yet.
        recommendations_df = self.popularity_df[~self.popularity_df['contentId'].isin(items_to_ignore)] \
                               .sort_values('eventStrength', ascending = False) \
                               .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(\
                                    self.items_df, \
                                    how = 'left', 
                                    left_on = 'contentId', 
                                    right_on = 'contentId')[['eventStrength', 'contentId', 'title', 'url', 'lang']]


        return recommendations_df
    
popularity_model = PopularityRecommender(item_popularity_df, articles_df)

In [13]:
popularity_model.recommend_items(-9223121837663643404)

,contentId,eventStrength
0,-4029704725707465084,307.733799
1,-6783772548752091658,233.762157
2,-133139342397538859,228.024567
3,-8208801367848627943,197.107608
4,-6843047699859121724,193.825208
5,8224860111193157980,189.044680
6,-2358756719610361882,183.110951
7,2581138407738454418,180.282876
8,7507067965574797372,179.094002
9,1469580151036142903,170.548969


In [14]:
print('Evaluating Popularity recommendation model...')
pop_global_metrics, pop_detailed_results_df = model_evaluator.evaluate_model(popularity_model)
print('\nGlobal metrics:\n%s' % pop_global_metrics)
pop_detailed_results_df.head(10)

Evaluating Popularity recommendation model...
1139 users processed

Global metrics:
{'modelName': 'Popularity', 'recall@5': 0.2418818716440808, 'recall@10': 0.3725389925850166}


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
76,28,50,192,0.145833,0.260417,3609194402293569455
17,12,25,134,0.089552,0.186567,-2626634673110551643
16,13,23,130,0.100000,0.176923,-1032019229384696495
10,5,9,117,0.042735,0.076923,-1443636648652872475
82,26,40,88,0.295455,0.454545,-2979881261169775358
161,12,18,80,0.150000,0.225000,-3596626804281480007
65,20,34,73,0.273973,0.465753,1116121227607581999
81,17,23,69,0.246377,0.333333,692689608292948411
106,14,18,69,0.202899,0.260870,-9016528795238256703
52,21,28,68,0.308824,0.411765,3636910968448833585


# Content-Based Filtering model
Content-based filtering approaches leverage description or attributes from items the user <br>
has interacted to recommend similar items. It depends only on the user previous choices, <br>
making this method robust to avoid the cold-start problem. For textual items, like articles, news and books, <br>
it is simple to use the raw text to build item profiles and user profiles.
Here we are using a very popular technique in information retrieval (search engines) named TF-IDF. <br>
This technique converts unstructured text into a vector structure, where each word is represented by<br> 
a position in the vector, and the value measures how relevant a given word is for an article.<br> 
As all items will be represented in the same Vector Space Model, it is to compute similarity between articles.
See this presentation (from slide 30) for more information on TF-IDF and Cosine similarity.

In [15]:
# nltk.download('stopwords')
#Ignoring stopwords (words with no semantics) from English and Portuguese (as we have a corpus with mixed languages)
stopwords_list = stopwords.words('english') + stopwords.words('portuguese')

#Trains a model whose vectors size is 5000, composed by the main unigrams and bigrams found in the corpus, ignoring stopwords
vectorizer = TfidfVectorizer(analyzer='word',
                     ngram_range=(1, 2),
                     min_df=0.003,
                     max_df=0.5,
                     max_features=5000,
                     stop_words=stopwords_list)

item_ids = articles_df['contentId'].tolist()
tfidf_matrix = vectorizer.fit_transform(articles_df['title'] + "" + articles_df['text'])
tfidf_feature_names = vectorizer.get_feature_names()
tfidf_matrix

<3047x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 638928 stored elements in Compressed Sparse Row format>

In [16]:
def get_item_profile(item_id):
    idx = item_ids.index(item_id)
    item_profile = tfidf_matrix[idx:idx+1]
    return item_profile

def get_item_profiles(ids):
    item_profiles_list = [get_item_profile(x) for x in ids]
    item_profiles = scipy.sparse.vstack(item_profiles_list)
    return item_profiles

def build_users_profile(person_id, interactions_indexed_df):
    interactions_person_df = interactions_indexed_df.loc[person_id]
    user_item_profiles = get_item_profiles(interactions_person_df['contentId'])
    
    user_item_strengths = np.array(interactions_person_df['eventStrength']).reshape(-1,1)
    #Weighted average of item profiles by the interactions strength
    user_item_strengths_weighted_avg = np.sum(user_item_profiles.multiply(user_item_strengths), axis=0) / np.sum(user_item_strengths)
    user_profile_norm = sklearn.preprocessing.normalize(user_item_strengths_weighted_avg)
    return user_profile_norm

def build_users_profiles(): 
    interactions_indexed_df = interactions_train_df[interactions_train_df['contentId'] \
                                                   .isin(articles_df['contentId'])].set_index('personId')
    user_profiles = {}
    for person_id in interactions_indexed_df.index.unique():
        user_profiles[person_id] = build_users_profile(person_id, interactions_indexed_df)
    return user_profiles

In [17]:
user_profiles = build_users_profiles()
# len(user_profiles)
cosine_similarities = cosine_similarity(user_profiles[-1479311724257856983], tfidf_matrix)
# Gets the top similar items
similar_indices = cosine_similarities.argsort().flatten()[-100:] # pandas.Series.argSort() returns the indices that sort the dataFrame.

# sorted([(1,2),(3,1),(7,0)], key=lambda x: x[1])
        

In [18]:
myprofile = user_profiles[-1479311724257856983]
print(myprofile.shape)
pd.DataFrame(sorted(zip(tfidf_feature_names, 
                        user_profiles[-1479311724257856983].flatten().tolist()), key=lambda x: -x[1])[:20],
             columns=['token', 'relevance'])

(1, 5000)


,token,relevance
0,learning,0.298732
1,machine learning,0.245992
2,machine,0.237843
3,google,0.202839
4,data,0.169776
5,ai,0.156203
6,algorithms,0.115666
7,like,0.097744
8,language,0.087609
9,people,0.082024


In [19]:
class ContentBasedRecommender:
    
    MODEL_NAME = 'Content-Based'
    
    def __init__(self, items_df=None):
        self.item_ids = item_ids
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def _get_similar_items_to_user_profile(self, person_id, topn=1000):
        #Computes the cosine similarity between the user profile and all item profiles
        cosine_similarities = cosine_similarity(user_profiles[person_id], tfidf_matrix)
        #Gets the top similar items
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        #Sort the similar items by similarity
        similar_items = sorted([(item_ids[i], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[1])
        return similar_items
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        similar_items = self._get_similar_items_to_user_profile(user_id)
        #Ignores items the user has already interacted
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        
        recommendations_df = pd.DataFrame(similar_items_filtered, columns=['contentId', 'recStrength']) \
                                    .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'contentId', 
                                                          right_on = 'contentId')[['recStrength', 'contentId', 'title', 'url', 'lang']]


        return recommendations_df
    
content_based_recommender_model = ContentBasedRecommender(articles_df)

print("The recommend items for user of person_Id -1479311724257856983 is :\n")
contentId=content_based_recommender_model.recommend_items(-1479311724257856983)["contentId"].tolist()
for id in contentId:
    print(articles_df[articles_df.contentId==id].title.values)

The recommend items for user of person_Id -1479311724257856983 is :

['How Google is Remaking Itself as a "Machine Learning First" Company - Backchannel']
['How Google is Remaking Itself as a "Machine Learning First" Company - Backchannel']
['Machine Learning for Designers']
['Machine Learning Is No Longer Just for Experts']
['Building AI Is Hard-So Facebook Is Building AI That Builds AI']
['How real businesses are using machine learning']
['5 Skills You Need to Become a Machine Learning Engineer | Udacity']
['Machine Learning as a Service: How Data Science Is Hitting the Masses']
['Is machine learning the next commodity?']
['Clarifying the uses of artificial intelligence in the enterprise']


In [20]:
print('Evaluating Content-Based Filtering model...')
cb_global_metrics, cb_detailed_results_df = model_evaluator.evaluate_model(content_based_recommender_model)
print('\nGlobal metrics:\n%s' % cb_global_metrics)
cb_detailed_results_df.head(10)

Evaluating Content-Based Filtering model...
1139 users processed

Global metrics:
{'modelName': 'Content-Based', 'recall@5': 0.16287394528253643, 'recall@10': 0.2614420864229097}


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
76,15,24,192,0.078125,0.125000,3609194402293569455
17,18,29,134,0.134328,0.216418,-2626634673110551643
16,20,33,130,0.153846,0.253846,-1032019229384696495
10,32,47,117,0.273504,0.401709,-1443636648652872475
82,6,15,88,0.068182,0.170455,-2979881261169775358
161,11,23,80,0.137500,0.287500,-3596626804281480007
65,8,13,73,0.109589,0.178082,1116121227607581999
81,8,19,69,0.115942,0.275362,692689608292948411
106,3,9,69,0.043478,0.130435,-9016528795238256703
52,3,8,68,0.044118,0.117647,3636910968448833585


# Collaborative filtering

In [21]:
users_items_pivot_matrix_df = interactions_train_df.pivot(index='personId', 
                                                          columns='contentId', 
                                                          values='eventStrength').fillna(0)

users_items_pivot_matrix_df.head(10)

contentId,-9222795471790223670,-9216926795620865886,-9194572880052200111,-9192549002213406534,-9190737901804729417,-9189659052158407108,-9176143510534135851,-9172673334835262304,-9171475473795142532,-9166778629773133902,...,9191014301634017491,9207286802575546269,9208127165664287660,9209629151177723638,9209886322932807692,9213260650272029784,9215261273565326920,9217155070834564627,9220445660318725468,9222265156747237864
personId,,,,,,,,,,,,,,,,,,,,,
-9223121837663643404,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-9212075797126931087,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-9207251133131336884,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-9199575329909162940,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-9196668942822132778,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-9188188261933657343,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-9172914609055320039,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-9156344805277471150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-9120685872592674274,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
users_items_pivot_matrix = users_items_pivot_matrix_df.values
users_items_pivot_matrix.shape

(1140, 2926)

In [23]:
users_ids=list(users_items_pivot_matrix_df.index)
users_ids[:10]


[-9223121837663643404,
 -9212075797126931087,
 -9207251133131336884,
 -9199575329909162940,
 -9196668942822132778,
 -9188188261933657343,
 -9172914609055320039,
 -9156344805277471150,
 -9120685872592674274,
 -9109785559521267180]

In [24]:
users_items_pivot_sparse_matrix = csr_matrix(users_items_pivot_matrix)
print(users_items_pivot_sparse_matrix)
# users_items_pivot_matrix.shape
users_items_pivot_matrix


  (0, 156)	1.0
  (0, 191)	1.0
  (0, 306)	3.1699250014423126
  (0, 320)	1.0
  (0, 376)	1.0
  (0, 431)	1.0
  (0, 439)	1.5849625007211563
  (0, 441)	1.5849625007211563
  (0, 453)	1.0
  (0, 686)	1.5849625007211563
  (0, 753)	1.0
  (0, 797)	1.0
  (0, 803)	1.5849625007211563
  (0, 846)	1.0
  (0, 871)	1.0
  (0, 1333)	1.0
  (0, 1365)	1.5849625007211563
  (0, 1385)	2.0
  (0, 1587)	1.0
  (0, 1601)	1.0
  (0, 1602)	1.0
  (0, 1674)	1.0
  (0, 1926)	1.0
  (0, 1954)	1.0
  (0, 1983)	1.0
  :	:
  (1139, 1617)	1.0
  (1139, 1681)	1.0
  (1139, 1811)	1.5849625007211563
  (1139, 1887)	1.5849625007211563
  (1139, 1901)	2.321928094887362
  (1139, 1914)	1.0
  (1139, 1938)	3.2479275134435857
  (1139, 2039)	1.5849625007211563
  (1139, 2146)	1.0
  (1139, 2288)	1.0
  (1139, 2303)	1.0
  (1139, 2404)	1.0
  (1139, 2423)	1.0
  (1139, 2491)	2.1699250014423126
  (1139, 2559)	1.0
  (1139, 2623)	2.1699250014423126
  (1139, 2642)	3.1699250014423126
  (1139, 2665)	1.0
  (1139, 2752)	1.0
  (1139, 2786)	1.0
  (1139, 2802)	3.247

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 2., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [91]:
# best=10
# checked=False
# np.random.seed(0)
# for val in range(10,100):
    
#     users_latent_features=np.random.rand(val,users_items_pivot_matrix.shape[0])
#     items_latent_features=np.random.rand(val,users_items_pivot_matrix.shape[1])
#     error=mean_squared_error(users_items_pivot_matrix,np.dot(users_latent_features.T,items_latent_features))
#     print(val,error)
#     if not checked:
#         min_error=error
#         checked=True
#     else:
#         if error<min_error:
#             best=val
# print(best)

    

In [86]:
# My code 
np.random.seed(0)
users_latent_features=np.random.rand(10,users_items_pivot_matrix.shape[0])
items_latent_features=np.random.rand(10,users_items_pivot_matrix.shape[1])
pred=np.dot(users_latent_features.T,items_latent_features)
pred=pd.DataFrame(pred,index=users_items_pivot_matrix_df.index,columns=users_items_pivot_matrix_df.columns).values
num_iterations=50
learning_rate=0.001
alpha=0.1

rows,cols=users_items_pivot_matrix.nonzero()
train_error=[]
m,n=users_items_pivot_matrix.shape
actual_sparse_mat=users_items_pivot_matrix_df.values

def rms(users_latent_features,items_latent_features,actual_sparse_mat):
    pred=np.dot(users_latent_features.T,items_latent_features)
    return mean_squared_error(pred,actual_sparse_mat)


start=time.time()
for _ in range(num_iterations):
    for row,col in zip(rows,cols):
        pred_val = np.dot(users_latent_features[:,row].T,items_latent_features[:,col])
        error = actual_sparse_mat[row,col]-pred_val
        # P=p+lr*(err*Q -alpha*P)
        users_latent_features[:,row] += learning_rate\
                                                * (error * items_latent_features[:,col]\
                                                - alpha \
                                                * users_latent_features[:,row])
        items_latent_features[:,col] += learning_rate\
                                                * (error * users_latent_features[:,row]\
                                                - alpha \
                                                * items_latent_features[:,col])
    MSE=rms(users_latent_features,items_latent_features,actual_sparse_mat)
    print(f"After {_} iterations the RMSE: ",np.sqrt(MSE))
end=time.time()

print("Total time taken to train: ",end-start)



After 0 iterations the RMSE:  2.464763910852572
After 1 iterations the RMSE:  2.39615186319173
After 2 iterations the RMSE:  2.3413942486209782
After 3 iterations the RMSE:  2.295703110900481
After 4 iterations the RMSE:  2.2564905197934624
After 5 iterations the RMSE:  2.222169508342113
After 6 iterations the RMSE:  2.1916851681963836
After 7 iterations the RMSE:  2.164295881678947
After 8 iterations the RMSE:  2.139458254048155
After 9 iterations the RMSE:  2.116761157957744
After 10 iterations the RMSE:  2.095885438230993
After 11 iterations the RMSE:  2.0765780504461056
After 12 iterations the RMSE:  2.05863479359319
After 13 iterations the RMSE:  2.041888396989323
After 14 iterations the RMSE:  2.0262000698689135
After 15 iterations the RMSE:  2.011453362228988
After 16 iterations the RMSE:  1.9975496110667772
After 17 iterations the RMSE:  1.984404500411823
After 18 iterations the RMSE:  1.9719454205434612
After 19 iterations the RMSE:  1.9601094115290658
After 20 iterations the 

In [87]:
best_predicted_matrix=np.dot(users_latent_features.T,items_latent_features)
best_predicted_matrix_scaled=(best_predicted_matrix - best_predicted_matrix.min())\
                    /(best_predicted_matrix.max()- best_predicted_matrix.min())
# best_predicted_matrix_scaled=scaler.fit_transform(best_predicted_matrix)
data_frame=pd.DataFrame(best_predicted_matrix_scaled,index=users_items_pivot_matrix_df.index,columns=users_items_pivot_matrix_df.columns)
data_frame

contentId,-9222795471790223670,-9216926795620865886,-9194572880052200111,-9192549002213406534,-9190737901804729417,-9189659052158407108,-9176143510534135851,-9172673334835262304,-9171475473795142532,-9166778629773133902,...,9191014301634017491,9207286802575546269,9208127165664287660,9209629151177723638,9209886322932807692,9213260650272029784,9215261273565326920,9217155070834564627,9220445660318725468,9222265156747237864
personId,,,,,,,,,,,,,,,,,,,,,
-9223121837663643404,0.247196,0.264395,0.221245,0.128996,0.188497,0.177735,0.169616,0.263977,0.169642,0.116073,...,0.180456,0.141632,0.133982,0.143074,0.157420,0.146130,0.195622,0.198914,0.159211,0.202313
-9212075797126931087,0.499628,0.382370,0.293988,0.286474,0.314706,0.332923,0.344310,0.356039,0.352446,0.200163,...,0.304526,0.316028,0.274098,0.342705,0.214403,0.243588,0.369970,0.376409,0.293934,0.339385
-9207251133131336884,0.459617,0.350416,0.321414,0.292719,0.351493,0.374906,0.289379,0.384113,0.280168,0.216859,...,0.347876,0.317045,0.329707,0.330107,0.215690,0.236002,0.408305,0.342243,0.274014,0.315437
-9199575329909162940,0.387174,0.329724,0.237022,0.197950,0.248836,0.234130,0.291377,0.270189,0.315290,0.155751,...,0.239360,0.241729,0.223705,0.291279,0.182467,0.212175,0.310848,0.304074,0.254556,0.278423
-9196668942822132778,0.260804,0.200378,0.260282,0.200826,0.305126,0.219410,0.132998,0.232414,0.206505,0.185054,...,0.211921,0.283134,0.373212,0.237060,0.093863,0.160913,0.331088,0.189958,0.177619,0.225063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9165571805999894845,0.464870,0.344524,0.306491,0.243448,0.302150,0.318060,0.320412,0.297370,0.345725,0.280114,...,0.289854,0.349350,0.299180,0.384381,0.168967,0.338932,0.420993,0.382175,0.306500,0.339657
9187866633451383747,0.355123,0.281415,0.259073,0.192225,0.256156,0.226174,0.202449,0.289621,0.216814,0.159977,...,0.253324,0.284943,0.239715,0.199341,0.143912,0.196520,0.246785,0.305419,0.160192,0.310914
9191849144618614467,0.495304,0.357350,0.354868,0.292534,0.353392,0.358267,0.336701,0.420812,0.396586,0.263907,...,0.249261,0.276884,0.269784,0.347324,0.254534,0.281114,0.477585,0.340218,0.348614,0.281140


In [64]:
# dict_form_df.columns=["index1","index2","ratings"]
# dict_form_df

In [65]:
#The number of factors to factor the user-item matrix.
NUMBER_OF_FACTORS_MF = 15
#Performs matrix factorization of the original user item matrix
#U, sigma, Vt = svds(users_items_pivot_matrix, k = NUMBER_OF_FACTORS_MF)
U, sigma, Vt = svds(users_items_pivot_sparse_matrix, k = NUMBER_OF_FACTORS_MF)
sigma=np.diag(sigma)

In [66]:
print(Vt.shape)
print(sigma.shape)
print(U.shape)


(15, 2926)
(15, 15)
(1140, 15)


In [67]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
all_user_predicted_ratings

array([[ 0.01039915,  0.00081872, -0.01725263, ...,  0.00140708,
         0.0110647 ,  0.00226063],
       [-0.00019285, -0.00031318, -0.00264624, ...,  0.00251658,
         0.00017609, -0.00189488],
       [-0.01254721,  0.0065947 , -0.00590676, ...,  0.00698975,
        -0.01015696,  0.01154572],
       ...,
       [-0.02995379,  0.00805715, -0.01846307, ..., -0.01083078,
        -0.00118591,  0.0096798 ],
       [-0.01845505,  0.00467019,  0.01219602, ...,  0.00409507,
         0.00019482, -0.00752562],
       [-0.01506374,  0.00327732,  0.13391269, ..., -0.01191815,
         0.06422074,  0.01303244]])

In [68]:
all_user_predicted_ratings_norm = (\
                                   all_user_predicted_ratings\
                                   - all_user_predicted_ratings.min())\
                                    / (all_user_predicted_ratings.max() \
                                    - all_user_predicted_ratings.min())

In [69]:
all_user_predicted_ratings_norm.shape

(1140, 2926)

In [70]:
# min_max_scaled=MinMaxScaler().fit_transform(all_user_predicted_ratings)


In [43]:
# min_max_scaled

In [71]:
#Converting the reconstructed matrix back to a Pandas dataframe
cf_preds_df = pd.DataFrame(all_user_predicted_ratings_norm,\
                           columns = users_items_pivot_matrix_df.columns,\
                           index=users_ids).transpose()
cf_preds_df.head(10)



,-9223121837663643404,-9212075797126931087,-9207251133131336884,-9199575329909162940,-9196668942822132778,-9188188261933657343,-9172914609055320039,-9156344805277471150,-9120685872592674274,-9109785559521267180,...,9105269044962898535,9109075639526981934,9135582630122950040,9137372837662939523,9148269800512008413,9165571805999894845,9187866633451383747,9191849144618614467,9199170757466086545,9210530975708218054
contentId,,,,,,,,,,,,,,,,,,,,,
-9222795471790223670,0.139129,0.137930,0.136531,0.143948,0.136815,0.137339,0.137508,0.143534,0.136428,0.135681,...,0.137351,0.127822,0.137946,0.139653,0.140324,0.136888,0.135787,0.134560,0.135862,0.136246
-9216926795620865886,0.138044,0.137916,0.138698,0.137878,0.137969,0.137990,0.137974,0.138049,0.138217,0.138151,...,0.137962,0.139527,0.138009,0.138117,0.139634,0.138058,0.138222,0.138864,0.138480,0.138323
-9194572880052200111,0.135998,0.137652,0.137283,0.137536,0.140363,0.137807,0.141246,0.136284,0.135301,0.138512,...,0.139257,0.143161,0.139139,0.140077,0.154976,0.140109,0.140654,0.135861,0.139332,0.153114
-9192549002213406534,0.141924,0.137996,0.134663,0.137080,0.139946,0.138574,0.139473,0.144469,0.143333,0.138428,...,0.140233,0.167426,0.138849,0.137037,0.141820,0.139260,0.139513,0.136804,0.140862,0.148793
-9190737901804729417,0.140209,0.137408,0.138708,0.138672,0.137725,0.138218,0.138390,0.138418,0.134883,0.140193,...,0.138373,0.138459,0.138169,0.137990,0.134041,0.137820,0.138100,0.138286,0.138630,0.136178
-9189659052158407108,0.138932,0.138699,0.138117,0.137621,0.138920,0.137766,0.138568,0.138200,0.140572,0.140365,...,0.140725,0.148152,0.137645,0.138165,0.149152,0.138912,0.139595,0.139807,0.140419,0.145698
-9176143510534135851,0.143208,0.138673,0.139514,0.139114,0.137664,0.137447,0.139833,0.140564,0.144698,0.144440,...,0.138367,0.146220,0.136204,0.138087,0.137317,0.137917,0.138546,0.142601,0.141431,0.142154
-9172673334835262304,0.138527,0.138021,0.138274,0.137827,0.137997,0.138037,0.138104,0.138259,0.137633,0.138397,...,0.138588,0.140146,0.138013,0.137839,0.137033,0.137969,0.138337,0.138361,0.138813,0.137538
-9171475473795142532,0.140720,0.137865,0.138061,0.137633,0.138231,0.138089,0.139009,0.137552,0.137143,0.140581,...,0.139046,0.139895,0.138000,0.137958,0.136061,0.138183,0.138817,0.138060,0.139205,0.137198


In [72]:
# # My conde

# dict_form=dict(users_items_pivot_sparse_matrix.todok().items())
# dict_form_rev_keys={}

# for index1,index2 in dict_form:
#     dict_form_rev_keys[(index2,index1)]=dict_form[(index1,index2)]
# del dict_form

# dict_form_rev_keys

# learning_rate=0.01
# error=0
# for (index1,index2) in dict_form_rev_keys:
#     error+=(dict_form_rev_keys[(index1,index2)]-cf_preds_df.iloc[index1,index2])**2


In [73]:
# RMSE=np.sqrt(error/cf_preds_df.shape[0])
# print("The root mean square root error is: ",RMSE)


In [74]:
len(cf_preds_df.columns)

1140

In [88]:

class CFRecommender:
    
    MODEL_NAME = 'Collaborative Filtering'
    
    def __init__(self, cf_predictions_df, items_df=None):
        self.cf_predictions_df = cf_predictions_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Get and sort the user's predictions
        sorted_user_predictions = self.cf_predictions_df[user_id].sort_values(ascending=False) \
                                    .reset_index().rename(columns={user_id: 'recStrength'})

        # Recommend the highest predicted rating movies that the user hasn't seen yet.
        recommendations_df = sorted_user_predictions[~sorted_user_predictions['contentId'].isin(items_to_ignore)] \
                               .sort_values('recStrength', ascending = False) \
                               .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'contentId', 
                                                          right_on = 'contentId')[['recStrength', 'contentId', 'title', 'url', 'lang']]


        return recommendations_df
    
cf_recommender_model = CFRecommender(data_frame.T, articles_df)

In [89]:
print('Evaluating Collaborative Filtering (SVD Matrix Factorization) model...')
cf_global_metrics, cf_detailed_results_df = model_evaluator.evaluate_model(cf_recommender_model)
print('\nGlobal metrics:\n%s' % cf_global_metrics)
cf_detailed_results_df.head(10)
print('Evaluating Collaborative Filtering (SVD Matrix Factorization) model...')
cf_global_metrics, cf_detailed_results_df = model_evaluator.evaluate_model(cf_recommender_model)
print('\nGlobal metrics:\n%s' % cf_global_metrics)
cf_detailed_results_df.head(10)

Evaluating Collaborative Filtering (SVD Matrix Factorization) model...
1139 users processed

Global metrics:
{'modelName': 'Collaborative Filtering', 'recall@5': 0.0219892610585528, 'recall@10': 0.04691894656098185}
Evaluating Collaborative Filtering (SVD Matrix Factorization) model...
1139 users processed

Global metrics:
{'modelName': 'Collaborative Filtering', 'recall@5': 0.0219892610585528, 'recall@10': 0.04691894656098185}


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
76,6,10,192,0.031250,0.052083,3609194402293569455
17,3,6,134,0.022388,0.044776,-2626634673110551643
16,7,12,130,0.053846,0.092308,-1032019229384696495
10,8,19,117,0.068376,0.162393,-1443636648652872475
82,1,3,88,0.011364,0.034091,-2979881261169775358
161,3,6,80,0.037500,0.075000,-3596626804281480007
65,1,2,73,0.013699,0.027397,1116121227607581999
81,3,8,69,0.043478,0.115942,692689608292948411
106,2,4,69,0.028986,0.057971,-9016528795238256703
52,0,1,68,0.000000,0.014706,3636910968448833585


In [ ]:
# #import numpy as np
# import pandas as pd
# train=np.array([
#         [1,1,1,0,1,1],
#         [1,1,1,2,1,1],
#         [1,1,1,2,1,1],
#         [1,1,1,2,1,1]
#     ])
# weight=np.array([1,2,3,1,1,2])
# weight=target.reshape(-1,1)
# predicted=np.dot(train,weight)+1 # bias is added
# target=np.array([1,1,2,2])
# # loss=target-predicted
# print(target)
# # print(loss)
# print(predicted)


In [ ]:
# import numpy as np
# import pandas as pd
# train=np.array([
#         [1,1,1,0,1,1],
#         [1,1,1,2,1,1],
#         [1,1,1,2,1,1],
#         [1,1,1,2,1,1]
#     ])
# weights=np.array([1,2,3,1,1,2])
# # weight=target.reshape(-1,1)

# print(loss)


# # print(predicted)


In [ ]:
# bias=0
# target=np.array([1,1,2,2])
# for i in range(100000):
#     predicted=np.dot(train,weights)+bias # bias is added
#     loss=target-predicted
#     derivative_of_weights=-2/train.shape[0] *(sum(train*(target-predicted)))
# #     derivative_of_bias=-2/train.shape[0] *(np.sum(target-predicted))
# #     weights= weights-0.001 * derivative_of_weights
# #     bias=bias-0.001*derivative_of_bias
#     print(weights,bias)

In [ ]:
derivative_of_weights

In [87]:
# df["score"]=df.groupby(["UserId","contentId"])["contentId"].transform('count')
# df

In [88]:
df=pd.DataFrame({"UserId":[-1902,-1903,1903,1905,1907,1956,1962,1702],"contentId":[123,123,123,456,678,908,908,685]})
df

,UserId,contentId
0,-1902,123
1,-1903,123
2,1903,123
3,1905,456
4,1907,678
5,1956,908
6,1962,908
7,1702,685


In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# corpus = [
#      'This is the first document.',
#     'This document is the second document.',
#      'And this is the third one.',
#      'Is this the first document?',
#  ]
# vectorizer = TfidfVectorizer()
# X = vectorizer.fit_transform(corpus)
# feature_names=vectorizer.get_feature_names()
# # print(feature_names)
# print(X)




# import pandas as pd
# dic={"userId":[1,1,1,2,3,4,9,6,8],"contentId":[123,234,456,789,574,623,798,985,610]}
# df=pd.DataFrame(dic)
# # print(df)
# df=df.groupby(["userId","contentId"]).sum().reset_index()
# df=df.set_index("userId")
# df.index.unique().values
# data=df.loc[1]["contentId"]
# list(indexed_test_df.index.unique().values)
# list(indexed_test_df.index.unique().values)
# print(type(data))
# data=set(data)
# data